In [1]:
import os
os.chdir('../')
%pwd

'/Users/pranavsrinivasvenkatesh/Projects/Data Science Projects/Kindey DIsease Classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/pranav10srinivas/Kindey-Disease-Classification-MLOPS-Project.mlflow"
os.environ["MLFLOW_TRACKING_REPO_OWNER"] = "pranav10srinivas"
os.environ["MLFLOW_TRACKING_REPO_NAME"] = "Kindey-Disease-Classification-MLOPS-Project"

In [6]:
#!pip install dagshub

In [39]:
import dagshub
dagshub.init(repo_owner='pranav10srinivas', repo_name='Kindey-Disease-Classification-MLOPS-Project', mlflow=True)



Initialized MLflow to track repo "pranav10srinivas/Kindey-Disease-Classification-MLOPS-Project"

Repository pranav10srinivas/Kindey-Disease-Classification-MLOPS-Project initialized!

In [7]:
import tensorflow as tf


In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


2025-11-24 15:19:43.441247: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-11-24 15:19:43.441473: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-24 15:19:43.441488: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-24 15:19:43.442003: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-24 15:19:43.442408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/pranav10srinivas/Kindey-Disease-Classification-MLOPS-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [28]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import shutil

In [ ]:
import mlflow.tensorflow


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)

        with mlflow.start_run():
            mlflow.set_tag("model_name", "KidneyDiseaseClassifier")
            # Log params and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # ================================
            # Export model using model.export()
            # ================================
            model_dir = "saved_model"
            
            # Delete previous directory
            if os.path.exists(model_dir):
                shutil.rmtree(model_dir)

            # Keras 3 → export SavedModel format
            self.model.export(model_dir)

            # ================================
            # Zip SavedModel directory
            # ================================
            zip_path = shutil.make_archive("saved_model", "zip", model_dir)

            # ================================
            # Log artifact to DAGSHub MLflow
            # ================================
            mlflow.log_artifact(zip_path)

In [40]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


/opt/miniconda3/envs/kidney/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 282ms/step - accuracy: 0.4283 - loss: 1.3800
INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')]
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  14055117392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14055114528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14054748224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14054749456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14055014064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14055012656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14055011776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14054743904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14054740912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14055142144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14054610896: TensorSpec(shape=(), dtype=tf.resource, n